In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix,bmat
import gc

from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import GroupShuffleSplit

import xgboost as xgb

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
cats = ['Green','Fairway','Intermediate Rough','Primary Rough','Fringe','Bunker','Other']

In [22]:
import pickle

In [121]:
with open('results.pkl','w') as pickleFile:
    pickle.dump(results,pickleFile)

In [96]:
def sum_minus_ind(arr):
    sums = [0]*len(arr)
    cumsum = 0
    for i in xrange(len(arr)):
        sums[i] = cumsum
        cumsum += arr[i]
        
    cumsum = 0
    for i in xrange(len(arr)-1,-1,-1):
        sums[i] += cumsum
        cumsum += arr[i]
    
    return np.array(sums)

In [97]:
sse = 0
grouped = data.groupby(['Hole','Round','Course_#','Year'])
print len(grouped)
for u,(tup,df) in enumerate(grouped):
    if u%1000==0:
        print u
    mean = sum_minus_ind(df.Shots_taken_from_location.values)/(len(df) - 1.0)
    sse += ((df.Shots_taken_from_location.values-mean)**2).sum()

38519
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


In [98]:
sse/(1.0*len(data))

0.36200629442119892

In [120]:
params = {'objective':'reg:linear','silent':1,'eta':.2,'max_depth':5,'subsample':.7,'min_child_weight':10,
          'alpha':.2,'lambda':1.5}
#results = {}
for cat in cats[1:]+['Tee Box']:
    print '************************* DOING %s *************************' % (cat,)
    
    data = pd.concat([pd.read_csv('./../new_data/%d.csv' % year)[['Cat','Shots_taken_from_location','Started_at_Z',
            'Distance_from_hole','Hole','Round','Course_#','Year','Green_to_work_with','Player_#']] for year in range(2003,2017)])
    data = data[data.Cat==cat]
    data.insert(len(data.columns),'Year-Course',data.Year.astype(str).str.cat(data['Course_#'].astype(str),sep='-'))
    data.insert(len(data.columns),'Hole-Course',data.Hole.astype(str).str.cat(data['Course_#'].astype(str),sep='-'))
    data.insert(len(data.columns),'Hole-Year-Course',data.Hole.astype(str).str.cat(
                                        [data.Year.astype(str),data['Course_#'].astype(str)],sep='-'))
    data.insert(len(data.columns),'Round-Year-Course',data.Round.astype(str).str.cat(
                                        [data.Year.astype(str),data['Course_#'].astype(str)],sep='-'))
    data.insert(len(data.columns),'Hole-Round-Year-Course',data.Hole.astype(str).str.cat(
                            [data.Round.astype(str),data.Year.astype(str),data['Course_#'].astype(str)],sep='-'))

    groups = ['-'.join(map(str,tup)) for tup in data[['Hole','Round','Course_#','Year','Player_#']].values.tolist()]
    le = LabelEncoder()
    groups = le.fit_transform(groups)

    y = data.Shots_taken_from_location.values
    
    lb = LabelBinarizer(sparse_output=True)
    results[cat] = {}
    
    ##simplest
    
    if cat=='Green': inds = [2,3]
    else: inds = [2,3,8]
    X = csr_matrix(data.values[:,np.array(inds)].astype(float))
    results[cat][0] = get_best_estimators(params,X,y,groups)

    ## with course

    X = bmat([[X,lb.fit_transform(data.values[:,6].astype(str))]],format='csr')
    results[cat][1] = get_best_estimators(params,X,y,groups)

    ## with year-course

    X = bmat([[X,lb.fit_transform(data.values[:,10].astype(str))]],format='csr')
    results[cat][2] = get_best_estimators(params,X,y,groups)

    ## with hole-course

    X = bmat([[X,lb.fit_transform(data.values[:,11].astype(str))]],format='csr')
    results[cat][3] = get_best_estimators(params,X,y,groups)

    ## with round-year-course

    X = bmat([[X,lb.fit_transform(data.values[:,12].astype(str))]],format='csr')
    results[cat][4] = get_best_estimators(params,X,y,groups)

    # with hole-year-course

    X = bmat([[X,lb.fit_transform(data.values[:,13].astype(str))]],format='csr')
    results[cat][5] = get_best_estimators(params,X,y,groups)

    # with everything

    X = bmat([[X,lb.fit_transform(data.values[:,14].astype(str))]],format='csr')
    results[cat][6] = get_best_estimators(params,X,y,groups)

************************* DOING Fairway *************************
**** FOLD 0 ****
0.318874243337
**** FOLD 1 ****
0.316839496941
**** FOLD 2 ****
0.315491261267
**** FOLD 3 ****
0.316697114508
**** FOLD 4 ****
0.318342878939
**** FOLD 5 ****
0.317362042722
**** FOLD 6 ****
0.317575041847
**** FOLD 7 ****
0.318767619214
**** FOLD 8 ****
0.31714237945
**** FOLD 9 ****
0.317858850822
**** FOLD 10 ****
0.316389720407
**** FOLD 11 ****
0.317311914705
******** MEAN OF 12 FOLDS 0.317388 ********
**** FOLD 0 ****
0.317873605671
**** FOLD 1 ****
0.315427098868
**** FOLD 2 ****
0.314889035772
**** FOLD 3 ****
0.31544860473
**** FOLD 4 ****
0.314752668394
**** FOLD 5 ****
0.316424315388
**** FOLD 6 ****
0.315800366718
**** FOLD 7 ****
0.315458056282
**** FOLD 8 ****
0.316280135505
**** FOLD 9 ****
0.316135272259
**** FOLD 10 ****
0.315473775003
**** FOLD 11 ****
0.314739621669
******** MEAN OF 12 FOLDS 0.315725 ********
**** FOLD 0 ****
0.313855861957
**** FOLD 1 ****
0.313710406715
**** FOLD 2 

In [147]:
dtrain = xgb.DMatrix(X[:,3:],label=y)

In [148]:
params = {'objective':'reg:linear','silent':1,'eta':.2,'max_depth':5,'subsample':.7,'min_child_weight':10,
          'alpha':.2,'lambda':1.5}

In [154]:
get_best_estimators(params,X[:,3:],y,groups,cv_folds=5)

**** FOLD 0 ****


KeyboardInterrupt: 

In [149]:
get_best_estimators(params,X,y,groups,cv_folds=5)

**** FOLD 0 ****
0.379370429832
**** FOLD 1 ****
0.379073598751
**** FOLD 2 ****
0.380044619986
**** FOLD 3 ****
0.379576698856
**** FOLD 4 ****
0.380347970428
******** MEAN OF 12 FOLDS 0.379683 ********


(0.37968266357055402, 0.00045898236221743495)

In [145]:
0.601628**2

0.3619562503840001

.2 - 0.616264
.1 - 0.616995
.05 - 0.617517

In [150]:
len(bst.get_score())

7477

In [153]:
for key in sorted(bst.get_score(),key=bst.get_score().get,reverse=True)[0:50]:
    print key,bst.get_score()[key]

f432 76
f176 45
f12865 41
f12797 38
f13802 38
f13725 36
f13552 36
f13948 34
f14148 33
f13405 32
f14144 32
f12505 32
f121 31
f13097 30
f12546 30
f12326 30
f14123 30
f12310 30
f12466 30
f13708 29
f12317 29
f12757 28
f13154 28
f12016 28
f13557 28
f12746 28
f1995 28
f12758 27
f13653 27
f13304 27
f12459 27
f13177 27
f13514 27
f13181 27
f13683 27
f13721 27
f12064 26
f12944 26
f12963 26
f12616 26
f13746 26
f14084 26
f12123 26
f14056 25
f13582 25
f618 25
f13487 25
f12470 25
f12634 25
f11999 25


In [ ]:
feature_names = ['elev_change','distance'] + ['Course = ' + name for name in lbins[6].classes_] + \
                ['Year = ' + name.strip().split('-')[0] + ', Course = ' + name.strip().split('-')[1] for name in lbins[9].classes_] + \
                ['Hole = ' + name.split('-')[0] + ', Course = ' + name.split('-')[1] for name in lbins[10].classes_] + \
                ['Hole = ' + name.split('-')[0] + ', Year = ' + name.split('-')[1] + ', Course = ' + name.split('-')[2] for name in lbins[11].classes_] + \
                ['Round = ' + name.split('-')[0] + ', Year = ' + name.split('-')[1] + ', Course = ' + name.split('-')[2] for name in lbins[12].classes_] + \
                ['Hole = ' + name.split('-')[0] + ', Round = ' + name.split('-')[1] + ', Year = ' + name.split('-')[2] + ', Course = ' + name.split('-')[3] for name in lbins[13].classes_]

In [ ]:
d = {(feature_names[int(key[1:])],int(key[1:])):value for key,value in bst.get_score().iteritems()}
df = pd.DataFrame({'label': [key[0] for key in d.iterkeys()],'ind': [key[1] for key in d.iterkeys()]
                   ,'count': [d[key] for key in d.iterkeys()]})
d = {feature_names[int(key[1:])]:(int(key[1:]),value) for key,value in bst.get_score().iteritems()}

In [115]:
def get_best_estimators(params,X,y,groups,cv_folds=12,early_stopping_rounds=50):
    cv = GroupShuffleSplit(n_splits=cv_folds, test_size=0.2)
    errors = []
    for u,(train,test) in enumerate(cv.split(X,y,groups)):
        print '**** FOLD %d ****' % (u,)
        dtrain = xgb.DMatrix(X[train],label=y[train])
        dtest = xgb.DMatrix(X[test],label=y[test])
        watchlist  = [(dtrain,'train'),(dtest,'eval')]
        num_round = 100000
        bst = xgb.train(params,dtrain,num_round,watchlist,early_stopping_rounds=early_stopping_rounds,verbose_eval=False)       
        error = np.mean((bst.predict(dtest,ntree_limit=bst.best_iteration) - y[test])**2)
        print error
        errors.append(error)
    print '******** MEAN OF %d FOLDS %g ********' % (cv_folds,np.array(errors).mean())
    return (np.array(errors).mean(),np.array(errors).std())

In [ ]:
len(bst.get_score())

In [ ]:
len(feature_names)

In [ ]:
df.sort_values('count',ascending=False).head(10).reset_index(drop=True)

In [ ]:
arr = np.zeros(52674)
arr[0] = -1
arr[1] = 10
arr = xgb.DMatrix(csr_matrix(arr),feature_names = dMat.feature_names)
bst.predict(arr)

In [ ]:
try: print d['Hole = 18, Round = 4, Year = 2004, Course = 512']
except: print 'none'
try: print d['Round = 4, Year = 2004, Course = 512']
except: print 'none'
try: print d['Hole = 18, Course = 512']
except: print 'none'
try: print d['Year = 2004, Course = 512']
except: print 'none'
try: print d['Course = 512']
except: print 'none'

In [ ]:
preds1 = []
preds2 = []
for dist in np.linspace(0.01,100,350):
    arr = np.zeros(52674)
    arr[1] = dist
    arr[0] = 0
    preds1.append(bst.predict(xgb.DMatrix(csr_matrix(arr),feature_names = dMat.feature_names)).tolist()[0])
    arr[35131] = 1
    arr[13667] = 1
    arr[1423] = 1
    arr[24] = 1
    preds2.append(bst.predict(xgb.DMatrix(csr_matrix(arr),feature_names = dMat.feature_names)).tolist()[0])

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(np.linspace(0.01,100,350)[0:30],preds1[0:30],label='generic');
plt.plot(np.linspace(0.01,100,350)[0:30],preds2[0:30],label='special green');
plt.legend();

In [ ]:
np.array(preds1)-np.array(preds2)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, subsample=.6, 
                                criterion='friedman_mse', min_samples_split=20, min_samples_leaf=5, 
                                max_depth=3, verbose=5)

In [ ]:
arr = np.zeros(52674)
arr[0] = 0
arr[1] = 1
gbr.predict(np.matrix(arr))

In [ ]:
(gbr.feature_importances_!=0).sum()

In [ ]:
for ind in np.argsort(gbr.feature_importances_)[::-1][:25]:
    print ind,dMat.feature_names[ind],gbr.feature_importances_[ind]

In [ ]:
gbr.fit(X,y)

In [ ]:
preds1 = []
preds2 = []
for dist in np.linspace(0,100,100):
    arr = np.zeros(52674)
    arr[1] = dist
    arr[0] = -.1
    preds1.append(gbr.predict(np.matrix(arr)))
    arr[0] = .1
    preds2.append(gbr.predict(np.matrix(arr)))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(np.linspace(0,100,100),preds1,label='uphill 1 ft.');
plt.plot(np.linspace(0,100,100),preds2,label='downhill 1 ft.');
plt.legend();

In [ ]:
for arg,num in zip(np.argsort(gbr.feature_importances_)[::-1],np.sort(gbr.feature_importances_)[::-1])[0:7]:
    print arg,num

In [ ]:
fig,axs = plot_partial_dependence(gbr,X_train_m2,[4],feature_names=,n_jobs=1,grid_resolution=50)

In [ ]:
np.mean(((lcv.predict(normalizer.transform(X_test)) + gbr.predict(X_test_m))/2 - y_test)**2)

In [ ]:
(((data.Cat=="Green").sum()*0.11545806607753986 +
 (data.Cat=="Fairway").sum()*0.31740937289 +
 (data.Cat=="Intermediate Rough").sum()*0.300202209856 +
 (data.Cat=="Primary Rough").sum()*0.361347962243 +
 (data.Cat=="Fringe").sum()*0.182508595153 + 
 (data.Cat=="Bunker").sum()*0.372257282426 +
 (data.Cat=="Other").sum()*0.493646743899)/len(data))**.5